In [7]:
import os, json, math, requests
from typing import Dict, Any, List, Optional
from google import genai
from google.genai import types

In [12]:
PROJECT_ID = "focus-surfer-468103-s3"   # ganti jika perlu
LOCATION   = "us-central1"              # JANGAN "global" untuk Vertex AI
MODEL_NAME = "gemini-2.5-flash"         # jika belum tersedia di regionmu, ganti "gemini-2.0-flash"
ADDRESS   = "130 Quang Trung, Hải Châu, Đà Nẵng, Vietnam"
RADIUS_M  = 300

# ================== OSM HELPERS ==================
NOMINATIM = "https://nominatim.openstreetmap.org/search"
OVERPASS  = "https://overpass-api.de/api/interpreter"

OVERPASS_QL = """
[out:json][timeout:25];
(
  node(around:{r},{lat},{lon})["amenity"~"hospital|clinic|police|fire_station|school|university|place_of_worship|bus_station"];
  node(around:{r},{lat},{lon})["public_transport"~"platform|stop_position|stop_area"];
  way(around:{r},{lat},{lon})["highway"];
  way(around:100,{lat},{lon})["highway"]["junction"="intersection"];
  way(around:{r},{lat},{lon})["access"="no"];
  way(around:{r},{lat},{lon})["busway"];
  way(around:100,{lat},{lon})["amenity"="parking"];
);
out body; >; out skel qt;
"""

HEADERS_OSM = {
    "User-Agent": "ParkLens-Research/1.0 (contact: you@example.com)"  # update kontakmu
}

In [13]:
def haversine_m(lat1, lon1, lat2, lon2):
    R=6371000
    from math import radians, sin, cos, asin, sqrt
    p1, p2 = radians(lat1), radians(lat2)
    dphi = radians(lat2-lat1); dl = radians(lon2-lon1)
    a = sin(dphi/2)**2 + cos(p1)*cos(p2)*sin(dl/2)**2
    return 2*R*asin(sqrt(a))

def geocode_address(address: str) -> Optional[Dict[str, float]]:
    params = {"q": address, "format": "json", "limit": 1}
    r = requests.get(NOMINATIM, params=params, headers=HEADERS_OSM, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not data:
        return None
    return {"lat": float(data[0]["lat"]), "lon": float(data[0]["lon"])}

def fetch_overpass(lat: float, lon: float, radius_m: int) -> Dict[str, Any]:
    q = OVERPASS_QL.format(lat=lat, lon=lon, r=radius_m)
    r = requests.post(OVERPASS, data={"data": q}, headers=HEADERS_OSM, timeout=60)
    r.raise_for_status()
    return r.json()

def transform_overpass(raw: Dict[str, Any], lat: float, lon: float, radius_m: int) -> Dict[str, Any]:
    elements = raw.get("elements", [])
    nodes = {e["id"]: e for e in elements if e["type"] == "node"}
    ways  = [e for e in elements if e["type"] == "way"]

    roads: List[Dict[str, Any]] = []
    for w in ways:
        tags = w.get("tags", {})
        if "highway" not in tags:
            continue
        n0 = nodes.get(w["nodes"][0]) if w.get("nodes") else None
        dist = round(haversine_m(lat, lon, n0["lat"], n0["lon"])) if n0 else None
        def to_int(v):
            try: return int(v)
            except: return 0
        roads.append({
            "name": tags.get("name"),
            "highway": tags.get("highway"),
            "maxspeed": to_int(tags.get("maxspeed", 0)),
            "lanes": to_int(tags.get("lanes", 0)),
            "distance_m": dist
        })

    pois: List[Dict[str, Any]] = []
    for e in elements:
        if e["type"] != "node": 
            continue
        t = e.get("tags", {})
        typ = (t.get("amenity") or t.get("public_transport") or "").lower()
        if typ in ["hospital","clinic","police","fire_station","school","university","bus_station","place_of_worship"]:
            d = round(haversine_m(lat, lon, e["lat"], e["lon"]))
            pois.append({"type": typ, "name": t.get("name"), "distance_m": d})

    # parking flag sederhana
    has_parking_lt50 = False
    for w in ways:
        tags = w.get("tags", {})
        if tags.get("amenity") == "parking" and w.get("nodes"):
            n0 = nodes.get(w["nodes"][0])
            if n0:
                d = haversine_m(lat, lon, n0["lat"], n0["lon"])
                if d < 50:
                    has_parking_lt50 = True
                    break

    features = {
        "roads": roads[:40],
        "pois": sorted(pois, key=lambda x: x["distance_m"])[:40],
        "intersections": [],  # bisa disempurnakan nanti
        "has_parking_lt50m": has_parking_lt50,
        "meta": {"radius_m": radius_m, "provider": "overpass"}
    }
    return features

In [16]:
# ================== LLM (Vertex AI via google-genai) ==================
def get_vertex_client():
    # Memakai ADC (Application Default Credentials)
    # Pastikan: gcloud auth application-default login  & Vertex AI API enabled
    return genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

def build_system_instruction() -> str:
    return (
        "You are an urgency arbiter for illegal parking events in the ParkLens AI system.\n"
        "1) Input: an event {cam_id, lat, lon, duration_s} and OSM-derived features.\n"
        "2) Compute a deterministic base_score (0–100) using this rubric:\n"
        "   - Roads: trunk +15, primary +12, secondary +8; maxspeed ≥60 +6; lanes ≥4 +4; intersection degree ≥4 +5.\n"
        "   - POI: hospital +20 (cap 40), clinic +12, fire_station +12, police +8,\n"
        "          school/university +8, bus_station +10, government +6, place_of_worship +5.\n"
        "   - Special: access=no or busway +8; <30 m from intersection +5.\n"
        "   - Counter: official parking <50 m −10; mall-dominant without arterial −6.\n"
        "   Clamp to 0–100.\n"
        "3) Optionally adjust by at most ±10 for critical factors explicitly supported by features.\n"
        "4) Output strictly in JSON per the schema; do not invent places. If features are partial, set confidence='medium'."
    )

def build_response_schema():
    # Schema untuk structured output (google-genai)
    return {
        "type":"OBJECT",
        "properties":{
            "urgency_score":{"type":"INTEGER","minimum":0,"maximum":100},
            "adjustment_reason":{"type":"STRING"},
            "reasons":{"type":"ARRAY","items":{"type":"STRING"},"maxItems":5},
            "narrative":{"type":"STRING"},
            "recommended_actions":{"type":"ARRAY","items":{"type":"STRING"},"maxItems":3},
            "confidence":{"type":"STRING","enum":["low","medium","high"]},
            "base_breakdown":{"type":"OBJECT","description":"Optional scoring breakdown keyed by factor"}
        },
        "required":["urgency_score","narrative","reasons","confidence"]
    }

def score_event_with_vertex(event: Dict[str,Any], features: Dict[str,Any]) -> Dict[str,Any]:
    client = get_vertex_client()
    system_text = build_system_instruction()

    contents = [
        types.Content(role="user", parts=[
            types.Part.from_text(text=system_text),
            types.Part.from_text(text=json.dumps({
                "event": event,
                "features": features
            }, ensure_ascii=False))
        ])
    ]

    cfg = types.GenerateContentConfig(
        temperature=0.1,
        top_p=0.95,
        seed=0,
        max_output_tokens=1024,
        response_mime_type="application/json",
        response_schema=build_response_schema(),
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
        thinking_config=types.ThinkingConfig(thinking_budget=0),
    )

    resp = client.models.generate_content(
        model=MODEL_NAME,
        contents=contents,
        config=cfg
    )
    try:
        return json.loads(resp.text)
    except Exception:
        return {"_raw": resp.text}


In [17]:
def main():
    # 1) Geocode
    loc = geocode_address(ADDRESS)
    if not loc:
        raise SystemExit(f"Gagal geocode: {ADDRESS}")
    lat, lon = loc["lat"], loc["lon"]
    print(f"[Geocode] {ADDRESS} -> lat={lat:.6f}, lon={lon:.6f}")

    # 2) Overpass OSM
    print("[Overpass] fetching…")
    raw = fetch_overpass(lat, lon, RADIUS_M)
    features = transform_overpass(raw, lat, lon, RADIUS_M)
    print(f"[Overpass] roads={len(features['roads'])}, pois={len(features['pois'])}, has_parking_lt50m={features['has_parking_lt50m']}")

    # 3) Event contoh
    event = {
        "cam_id": "CCTV-DN-QUANGTRUNG-01",
        "lat": lat,
        "lon": lon,
        "duration_s": 128
    }

    # 4) Scoring via Vertex AI
    result = score_event_with_vertex(event, features)
    print("\n=== Urgency JSON ===")
    print(json.dumps(result, ensure_ascii=False, indent=2))

if __name__ == "__main__":
    # Pastikan: gcloud auth application-default login  & Vertex AI API enabled
    main()

[Geocode] 130 Quang Trung, Hải Châu, Đà Nẵng, Vietnam -> lat=16.074303, lon=108.216562
[Overpass] fetching…
[Overpass] roads=40, pois=4, has_parking_lt50m=False


c:\Users\andre\anaconda3\envs\datsci\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)



=== Urgency JSON ===
{
  "urgency_score": 60,
  "adjustment_reason": "No critical factors for adjustment.",
  "reasons": [
    "Proximity to hospital",
    "Proximity to school",
    "Secondary road nearby"
  ],
  "narrative": "The parking event occurred near a hospital and a school, increasing its urgency. The closest major road is a secondary road.",
  "recommended_actions": [
    "Issue citation",
    "Dispatch traffic control"
  ],
  "confidence": "medium",
  "base_breakdown": {}
}
